In [1]:
# Install required dependencies
!pip install ultralytics opencv-python torch torchvision torchaudio pandas numpy tensorflow scikit-learn fastapi uvicorn pymongo

import cv2
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO
from pymongo import MongoClient
from fastapi import FastAPI
import uvicorn


   ---------------------------------------- 0.0/921.9 kB ? eta -:--:--
   - -------------------------------------- 41.0/921.9 kB ? eta -:--:--
   -- ------------------------------------- 61.4/921.9 kB 3.4 MB/s eta 0:00:01
   -- ------------------------------------- 61.4/921.9 kB 3.4 MB/s eta 0:00:01
   -- ------------------------------------- 61.4/921.9 kB 3.4 MB/s eta 0:00:01
   -------- ----------------------------- 194.6/921.9 kB 908.0 kB/s eta 0:00:01
   -------- ----------------------------- 194.6/921.9 kB 908.0 kB/s eta 0:00:01
   --------- ---------------------------- 235.5/921.9 kB 801.7 kB/s eta 0:00:01
   ------------ ------------------------- 307.2/921.9 kB 865.8 kB/s eta 0:00:01
   ------------- ------------------------ 337.9/921.9 kB 840.2 kB/s eta 0:00:01
   --------------- ---------------------- 368.6/921.9 kB 820.1 kB/s eta 0:00:01
   ----------------- -------------------- 419.8/921.9 kB 904.2 kB/s eta 0:00:01
   ------------------ ------------------- 450.6/921.9 kB 829

In [5]:
# Initialize FastAPI app for web integration
app = FastAPI()

In [7]:
# Load the best YOLOv8 model for high accuracy
model = YOLO('yolov8x.pt')  # Extra-large model for precision

100%|████████████████████████████████████████████████████████████████████████████████| 131M/131M [02:57<00:00, 770kB/s]


In [9]:
# Connect to MongoDB for storing detections
client = MongoClient("mongodb://localhost:27017/")
db = client["RailwayDetection"]
collection = db["Detections"]

In [11]:
# Define railway features to detect based on PDFs
railway_features = [
    "OHE-Pole", "Traffic Signal Pole", "Gradient Post", "Hectometer Post", "Sign Pole", "Lamp Post",
    "Electrical Pole", "Station Name Board", "Bridge Details Board", "Curve Details Board", "Turnout Board",
    "LWR Board", "TI Board", "Tractive Effort Board", "W-Board", "Catch Water Drain", "Fence", "Height Gauge",
    "Trolley Refuge", "Tunnel", "Wall", "Silver Box", "Rail Track", "Platform", "Railway Shed", "RoB", "RuB",
    "Flag at Railway Station", "FoB", "Land Boundary Stone", "KM.Stone", "Chainage Pole"
]

In [13]:
# Define video path
video_path = "railway_video.mp4"
cap = cv2.VideoCapture(video_path)

In [ ]:
# Check if video opened successfully
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

In [ ]:
# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
# Define output file
output_path = "processed_video.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [ ]:
# List to store detections
detections = []
frame_count = 0

In [ ]:
# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    timestamp = frame_count / fps  # Time in seconds

    # Preprocessing: Improve frame quality for better detection
    frame = cv2.GaussianBlur(frame, (5, 5), 0)  # Reduce noise
    frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=30)  # Enhance contrast

    # Object detection with higher confidence threshold
    results = model(frame, conf=0.5, iou=0.4)  # Higher confidence for better accuracy
    for result in results:
        for box, cls, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            label = model.names[int(cls)]
            confidence = float(conf)
            
            if label in railway_features and confidence > 0.7:  # Ensure high accuracy detections
                detections.append([frame_count, timestamp, label, confidence, x1, y1, x2, y2])
                
                # Store detection in MongoDB
                detection_record = {
                    "Frame": frame_count,
                    "Time": timestamp,
                    "Object": label,
                    "Confidence": confidence,
                    "Coordinates": [x1, y1, x2, y2]
                }
                collection.insert_one(detection_record)
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{label} ({confidence:.2f})', (x1, y1 - 5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Write processed frame
    out.write(frame)
    
    # Display the frame
    cv2.imshow("Railway Feature Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Save results to CSV
df = pd.DataFrame(detections, columns=["Frame", "Time", "Object", "Confidence", "X1", "Y1", "X2", "Y2"])
df.to_csv("railway_detections.csv", index=False)

In [ ]:
# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete. High-accuracy results saved.")


In [ ]:
# FastAPI route to get detections
@app.get("/detections")
def get_detections():
    data = list(collection.find({}, {"_id": 0}))
    return {"detections": data}

In [ ]:
# Run FastAPI server
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
